In [ ]:
import sys
# This is the path to where the sample plugin is built in the docker image.
# Adjust as necessary.
sys.path.append('/home/jovyan/plugin-build/src/pythonmodule')

In [ ]:
import gmx
import myplugin

In [ ]:
help(gmx)

In [ ]:
help(myplugin)

In [ ]:
from gmx.data import tpr_filename
system = gmx.System._from_file(tpr_filename)
potential = myplugin.HarmonicRestraint()
potential.set_params(1, 4, 2.0, 100.0)
# potential.set_params(1, 4, 0, 0)

system.add_potential(potential)
with gmx.context.DefaultContext(system.workflow) as session:
    session.run()

In [ ]:
md = gmx.workflow.from_tpr([tpr_filename, tpr_filename])
print(md.workspec)

In [ ]:
print(md.serialize())

In [ ]:
potential = gmx.workflow.WorkElement(namespace="myplugin",
                                     operation="create_restraint",
                                     params=[1, 4, 2.0, 10000.0])
potential.name = "harmonic_restraint"
md.add_dependency(potential)

In [ ]:
print(md.serialize())

In [ ]:
print(md.workspec)

In [ ]:
context = gmx.context.ParallelArrayContext(md)
with context as session:
    if context.rank == 0:
        print(context.work)
    session.run()

In [ ]:
md = gmx.workflow.from_tpr(tpr_filename)
md.add_dependency(potential)


In [ ]:
potential.workspec = None
md.add_dependency(potential)

In [ ]:
context = gmx.context.ParallelArrayContext(md)
with context as session:
    if context.rank == 0:
        print(context.work)
    status = session.run()
print(status)